# Notebook to showcase how to use the model

## CLEAR OUTPUT OF THE NOTEBOOK BEFORE COMMITING/PUSHING

The path may not work for you as they are hardcoded for a sample of the data on my machine

In [ ]:
import torch
from torchvision.io import read_image
import torchvision.transforms as T
from cnn import CNN
from encoder import Encoder
import os
import csv
import re

In [ ]:
FOLDER_PATH = '../../data/CROHME2016_data/data_png/subset/'

In [ ]:
def sort_files(file):
    """Utility function to sort the file names according to their number"""
    match = re.match(r'\D*(?P<num>\d+)\..*', file)
    if match:
        return int(match.group('num'))   

In [ ]:
#loading the images in one tensor
batch = torch.zeros((11,1,304,304))
tree = next(os.walk(FOLDER_PATH))
files = [file for file in tree[-1] if file.endswith('png')]
files.sort(key=sort_files)
for i,file in enumerate(files):
    pic = read_image(f"{FOLDER_PATH}/{file}").to(torch.float32)
    batch[i] += pic

In [ ]:
#load the labels in a dictionary
labels = dict()
with open(f"{FOLDER_PATH}iso_GT.txt") as f:
    reader = csv.reader(f)
    for row in reader:
        labels[row[0]] = row[1]

In [ ]:
net = CNN()
encoder = Encoder(512, 256, 27*24, 11)
out = net(batch)
o, h = encoder(out, None, True)

In [ ]:
o.shape

In [ ]:
img = o.reshape((27,24, 11,256))[:,:,6,0]

In [ ]:
import matplotlib.pyplot as plt
#example of the feature grid F' for the first image
plt.imshow(img.detach().numpy(), origin='lower')

In [ ]:
#encoder without the linear layer at the end
o, h = encoder(out, None, False)
o.shape